# Supervision

conda create -n supervision python=3.11.5
conda activate supervision

Sobre VC_P4
pip install opencv-python
pip install supervision inference -q

In [ ]:
#Fuente https://roboflow.github.io/cheatsheet-supervision/
# 
import cv2
import supervision as sv

#https://supervision.roboflow.com/latest/how_to/track_objects/#tracking
#Ultralytics, modo inference para ejecutar en la  nube con una key

from supervision.assets import download_assets, VideoAssets

download_assets(VideoAssets.PEOPLE_WALKING)

import numpy as np
import supervision as sv
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
tracker = sv.ByteTrack()
box_annotator = sv.BoundingBoxAnnotator()
label_annotator = sv.LabelAnnotator()
trace_annotator = sv.TraceAnnotator()

def callback(frame: np.ndarray, _: int) -> np.ndarray:
    results = model(frame)[0]
    detections = sv.Detections.from_ultralytics(results)
    detections = tracker.update_with_detections(detections)

    labels = [
        f"#{tracker_id} {results.names[class_id]}"
        for class_id, tracker_id
        in zip(detections.class_id, detections.tracker_id)
    ]

    annotated_frame = box_annotator.annotate(
        frame.copy(), detections=detections)
    annotated_frame = label_annotator.annotate(
        annotated_frame, detections=detections, labels=labels)
    return trace_annotator.annotate(
        annotated_frame, detections=detections)

sv.process_video(
    source_path="people-walking.mp4",
    target_path="result.mp4",
    callback=callback
)


In [ ]:
Fútbol https://github.com/roboflow/sports/tree/main/examples/soccer